<a href="https://colab.research.google.com/github/Aarush49/RAG-Project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Dependencies

In [1]:
!pip install chromadb sentence-transformers pypdf google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 8.9 MB/s e

# Step 2: Upload and Extract text from PDF

In [2]:
from google.colab import files
from pypdf import PdfReader

uploaded = files.upload()
pdf_path = next(iter(uploaded))
reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
  text += page.extract_text() + "\n"

print(text[:1000])  # Preview first 1000 chars

Saving Lecture 1-11 Worksheet_copy.pdf to Lecture 1-11 Worksheet_copy.pdf
4 CONTENTS
Lecture 1: Pre-Calculus Review-I
The following formulas are very important:
(a)(𝑎+ 𝑏)2 = 𝑎2 + 2𝑎𝑏+ 𝑏2 (b)(𝑎− 𝑏)2 = 𝑎2 − 2𝑎𝑏+ 𝑏2
(c)(𝑎+ 𝑏)3 = 𝑎3 + 3𝑎2𝑏+ 3𝑎𝑏2 + 𝑏3 (d)(𝑎− 𝑏)3 = 𝑎3 − 3𝑎2𝑏+ 3𝑎𝑏2 − 𝑏3
(e)𝑎3 − 𝑏3 = (𝑎− 𝑏) (𝑎2 + 𝑎𝑏+ 𝑏2) (f)𝑎3 + 𝑏3 = (𝑎+ 𝑏)(𝑎2 − 𝑎𝑏+ 𝑏2)
(g)𝑎2 − 𝑏2 = (𝑎− 𝑏)(𝑎+ 𝑏)
Example 1.Factor Completely
(a) 𝑥3 − 27 =
(b) 𝑥3 + 125 =
Zero Product Rule: The fundamental idea behind solving quadratic equation is the following rule: For any real
numbers 𝑥and 𝑦:
𝑥𝑦= 0if and only if𝑥= 0or𝑦= 0.
Quadratic Equation: The solutions to𝑎𝑥2 + 𝑏𝑥+ 𝑐= 0, where𝑎≠ 0 are given by
𝑥= −𝑏±
√
𝑏2 − 4𝑎𝑐
2𝑎 .
Example 2.Solve (a)2𝑥2 − 5𝑥+ 2 = 0 (b)(2𝑥+ 1)(𝑥− 3) = −6
(c)(2𝑥+ 1)(𝑥− 3) = 0
CONTENTS 5
Definition and Properties of Absolute Value
|𝑥| =
{
𝑥 𝑖𝑓 𝑥 ≥ 0
−𝑥 𝑖𝑓 𝑥< 0
Properties of Absolute Value
1. |𝑥| ≥ 0
2. |𝑥| = 0iff 𝑥= 0
3. |𝑥𝑦| = |𝑥|⋅|𝑦|
4. 𝑥≤ |𝑥|
5. |𝑥± 𝑦| ≤ |𝑥|+ |𝑦|
6. For𝑎≥ 0,|𝑥| ≤ 𝑎if and only is−𝑎≤ 𝑥≤ 𝑎
7.

# Step 3: Chunk and Vectorize Text

In [3]:
from sentence_transformers import SentenceTransformer

CHUNK_SIZE = 500
chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks)
print(embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.08354105 -0.01346478  0.09308051 ...  0.11684061  0.03304186
   0.10508572]
 [-0.00302361  0.0015555   0.03850439 ...  0.05640342 -0.00706805
   0.03878733]
 [-0.05291754  0.00329352 -0.01658832 ...  0.08492494  0.0195426
   0.01152919]
 ...
 [-0.01080419 -0.0492948   0.09893639 ...  0.08765394  0.13700365
   0.02336822]
 [-0.06070526 -0.03594633  0.0822835  ...  0.12951662  0.05225106
   0.00305701]
 [-0.01616168  0.0482909   0.13569836 ...  0.03451607  0.03048179
  -0.02494321]]


# Step 4: Use ChromaDB for storing vectorized text

In [4]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.create_collection(name="pdf_notes")
ids = [f"chunk_{i}" for i in range(len(chunks))]
collection.add(documents = chunks, embeddings = embeddings.tolist(), ids = ids)


# Step 5: Take input and generate context

In [5]:
query = input("Enter your question: ")
query_emb = embedder.encode([query])[0]

results = collection.query(query_embeddings=[query_emb], n_results=3)

context = "\n---\n".join(results['documents'][0])
print("Context:\n", context)

Enter your question: (a+b)^2=?
Context:
 4 CONTENTS
Lecture 1: Pre-Calculus Review-I
The following formulas are very important:
(a)(𝑎+ 𝑏)2 = 𝑎2 + 2𝑎𝑏+ 𝑏2 (b)(𝑎− 𝑏)2 = 𝑎2 − 2𝑎𝑏+ 𝑏2
(c)(𝑎+ 𝑏)3 = 𝑎3 + 3𝑎2𝑏+ 3𝑎𝑏2 + 𝑏3 (d)(𝑎− 𝑏)3 = 𝑎3 − 3𝑎2𝑏+ 3𝑎𝑏2 − 𝑏3
(e)𝑎3 − 𝑏3 = (𝑎− 𝑏) (𝑎2 + 𝑎𝑏+ 𝑏2) (f)𝑎3 + 𝑏3 = (𝑎+ 𝑏)(𝑎2 − 𝑎𝑏+ 𝑏2)
(g)𝑎2 − 𝑏2 = (𝑎− 𝑏)(𝑎+ 𝑏)
Example 1.Factor Completely
(a) 𝑥3 − 27 =
(b) 𝑥3 + 125 =
Zero Product Rule: The fundamental idea behind solving quadratic equation is the following rule: For any real
numbers 𝑥and 𝑦:
𝑥𝑦= 0if and only 
---
g𝑎𝑥= ln 𝑥
ln 𝑎
Example 7.Use the properties of logarithms to expand the expression completely
ln
[
(𝑥2 + 3)4
𝑥
3√
𝑥2 + 1
]
.
Example 8.Graph the functions. Write the domain and range
(a)log2(𝑥+ 2) (b)log2(𝑥− 2) (c)log2(𝑥) + 2 (d)log2(𝑥) − 2
8 CONTENTS
Lecture 2: PreCalculus Review-II
Useful Trigonometric Identities
(a) sin2 (𝜃) + cos2 (𝜃) = 1
(b) 1 + tan2 𝜃= sec2 𝜃
(c) 1 + cot2 𝜃= csc2 𝜃
(d) sin (𝛼± 𝛽) = sin(𝛼) cos (𝛽) ± sin(𝛽) cos (𝛼)
(e) cos (𝛼± 𝛽)

# Step 6: Use Gemini to answer questions from the context

In [6]:
from google.colab import userdata
import google.generativeai as genai

# Make sure to have API_KEY in your secrets
genai.configure(api_key=userdata.get("API_KEY"))

model = genai.GenerativeModel('models/gemini-2.5-flash')

prompt = f"Answer the follwoing question using the context given. Context: {context} \n Question: {query}"

response = model.generate_content(prompt)
print("Answer:", response.text)

Answer: According to the context provided in Lecture 1: Pre-Calculus Review-I, the formula for (a+b)^2 is:

(a+ b)2 = a2 + 2ab+ b2
